# embedding 결과부터, 학습을 위한 ordered_embedding 산출까지

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import os
import pickle

%matplotlib inline

In [2]:
from sklearn import cluster, metrics

In [3]:
import networkx as nx
from collections import Counter

In [5]:
import pandas as pd
import glob
import os

In [6]:
import gensim

---

## # word2vec 이용
- 수많은 상품들이 있는 상황에서 유사도 살피기 위해

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format('result_data/embedding_w2v_1209.txt')

### embedding_1209.csv 도 필요하다
- 상품번호 sampling하려면 필요해

In [15]:
research_pid = [
 180224001,
 221118465,
 215220227,
 220069896,
 260702225,
 166854676,
 307363868,
 161611807,
 207093791,
 343801891,
 266469415,
 254672936,
 139853870
]

In [7]:
fields = ['id']

# prod_id_list = pd.read_csv('embedding_1209.csv', skipinitialspace=True, usecols=fields)
prod_embedding = pd.read_csv('embedding_1209.csv' , index_col=0)

/home/wmind/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
prod_embedding.head()

,id,lcate_cd,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,"602610157_쿠폰추가★11개구매시쿠폰가8,170원!한끼대용간편선식국내산100%...",1100004,0.446286,0.277313,0.177863,0.224614,0.053937,0.116937,0.243198,0.347476,...,0.297818,-0.004374,0.139615,0.317857,0.093901,0.317629,0.224229,0.299337,0.200292,0.232984
1,600515938_[실라리안]국내산돼지고기로만든가성비갑만두모음!,1100000,0.516052,0.281411,0.333511,0.254800,-0.006607,0.032997,0.148115,0.360973,...,0.360331,0.038403,0.039156,0.249721,0.167012,0.295624,0.055712,0.135611,0.170498,0.294120
2,600185944_[실라리안]100%우리콩안동낫또골라담기,1100000,0.417226,0.252049,0.292064,0.212674,0.004232,0.063358,0.146772,0.339002,...,0.345188,0.015622,0.104531,0.276849,0.135741,0.272882,0.071401,0.177105,0.180849,0.312720
3,"601529111_닥터딥민감성피부관리여드름성피부,아기피부,수부지피부",1100059,0.421385,0.288960,0.143851,0.133606,0.082439,0.099255,0.236265,0.350072,...,0.334989,0.111873,0.071139,0.361715,0.051260,0.204689,0.237831,0.210794,0.070109,0.124177
4,600016493_[쇼핑의모든것]대천갓바위김도시락김120봉가격특가,1100014,0.544923,0.384857,0.284561,0.176304,-0.009811,-0.000901,0.033510,0.195942,...,0.457690,-0.031100,0.037204,0.394122,0.054656,0.292350,-0.014870,0.095671,0.238219,0.325416


---

## # 상품 고유의 id 지정
- 상품 title은 중복될 수 있다(상품 고유번호와 상품명을 혼합하여 인식하자)
- id  : 상품 고유번호+상품명
- pid : 상품 고유번호

In [9]:
prod_id_list.head()

,id
0,"602610157_쿠폰추가★11개구매시쿠폰가8,170원!한끼대용간편선식국내산100%..."
1,600515938_[실라리안]국내산돼지고기로만든가성비갑만두모음!
2,600185944_[실라리안]100%우리콩안동낫또골라담기
3,"601529111_닥터딥민감성피부관리여드름성피부,아기피부,수부지피부"
4,600016493_[쇼핑의모든것]대천갓바위김도시락김120봉가격특가


In [13]:
prod_id_list['pid'] = prod_id_list['id'].apply(lambda x: x.split('_')[0])

In [17]:
prod_id_list['pid'] = prod_id_list['pid'].astype(int)

In [14]:
prod_id_list.head()

,id,pid
0,"602610157_쿠폰추가★11개구매시쿠폰가8,170원!한끼대용간편선식국내산100%...",602610157
1,600515938_[실라리안]국내산돼지고기로만든가성비갑만두모음!,600515938
2,600185944_[실라리안]100%우리콩안동낫또골라담기,600185944
3,"601529111_닥터딥민감성피부관리여드름성피부,아기피부,수부지피부",601529111
4,600016493_[쇼핑의모든것]대천갓바위김도시락김120봉가격특가,600016493


In [18]:
prod_id_list[prod_id_list.pid.isin(research_pid)]

,id,pid
5257018,307363868_HP프린터용32V625MAAC전원공급장치어댑터충전기Officej/...,307363868
5257218,215220227_[무료배송]등산촬영머리모자헬멧스마트폰헤드캠마운트,215220227
5322997,166854676_[MFG-500]마론프레스코이중유리컵,166854676
5336064,254672936_[호주건강식품]블랙모어스발레리안포르테2000mg30정/Blackm...,254672936
5360145,180224001_[투쿨포스쿨]히알루로닉선세럼SPF50+PA+++_50ml,180224001
5360652,161611807_[다이소]일본제디즈니아이스트레이M미키마우스-435516206,161611807
5364511,266469415_코코마트/스티커제거펜9ml스티커제거제스티커자국스티커자국제거스티커자...,266469415
5372070,260702225_한보장파워타임홍삼스틱10mlx30포,260702225
5413463,207093791_삼양튀김칼국수117g40입,207093791
5457624,221118465_(현대Hmall)GIVY기비R(8R6)메모리솔리드다운(GHW07R...,221118465


In [19]:
research_prod_list = prod_id_list[prod_id_list.pid.isin(research_pid)].id.tolist()

---

## # 조사 진행

In [20]:
research_prod_list

['307363868_HP프린터용32V625MAAC전원공급장치어댑터충전기Officej/910035',
 '215220227_[무료배송]등산촬영머리모자헬멧스마트폰헤드캠마운트',
 '166854676_[MFG-500]마론프레스코이중유리컵',
 '254672936_[호주건강식품]블랙모어스발레리안포르테2000mg30정/BlackmoresValerianForte2000mg',
 '180224001_[투쿨포스쿨]히알루로닉선세럼SPF50+PA+++_50ml',
 '161611807_[다이소]일본제디즈니아이스트레이M미키마우스-435516206',
 '266469415_코코마트/스티커제거펜9ml스티커제거제스티커자국스티커자국제거스티커자국제거제다용도스티커제거제',
 '260702225_한보장파워타임홍삼스틱10mlx30포',
 '207093791_삼양튀김칼국수117g40입',
 '221118465_(현대Hmall)GIVY기비R(8R6)메모리솔리드다운(GHW07R6)G7WH0R6-99',
 '220069896_크림빵이늘었다줄었다(456수학동화4:덧셈과뺄셈)(양장)',
 '139853870_남여공용루즈핏후드집업',
 '343801891_김치냉장고용폭조절2단선반리빙스타행거리빙스타행거선반행거2단행거']

In [21]:
for prod in research_prod_list:
    try:
        #print(prod)
        print('***  id : {}'.format( prod ))
        for similar in model.wv.most_similar( prod ):
            print(similar)
        print('\n\n')
    except:
        pass

***  id : 307363868_HP프린터용32V625MAAC전원공급장치어댑터충전기Officej/910035


/home/wmind/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


('306952431_[원더랜드]ASUSN56V용누빔노트북가방15.6W', 1.0000001192092896)
('211416403_독보적인유니크함파스텔투톤컬러맥북전용케이스', 1.0000001192092896)
('183906570_삼성AlwaysPen노트북9전원어댑터19V3.42A65W외경3.0mm호환일체형충전기', 1.0000001192092896)
('155024623_멀티폴드맥북노트북거치대NMF-H1', 1.0000001192092896)
('100206685_1515.4맥북노트북파우치가방', 1.0000001192092896)
('143716274_알럽스킨LG그램2in114T990종이질감액정보호필름', 1.0000001192092896)
('147531250_클러치백스타일노트북파우치맥북파우치13인치15인치', 1.0000001192092896)
('301703412_삼성노트북9메탈15인치고급파우치가방케이스', 1.0000001192092896)
('602657333_[라온]LG그램,맥북노트북슬림슬리브파우치', 1.0000001192092896)
('224852849_액정보호필름보안NT900X3G-K25SLCDPC아이티플러스AG노트북보호용품저반사아티브북9', 1.0)



***  id : 215220227_[무료배송]등산촬영머리모자헬멧스마트폰헤드캠마운트
('120866595_[정품]요이치욜로무선블루투스셀카봉삼각대WT300', 1.0000001192092896)
('100175631_요이치욜로무선블루투스셀카봉삼각대WT300', 1.0000001192092896)
('303448551_요이치글로우조명LED블루투스셀카봉:핑크', 1.0000001192092896)
('307347274_HD4K22x줌휴대전화망원경렌즈망원외부스마트폰카메라/1052674', 1.0000001192092896)
('100101893_디스포고급블루투스셀카봉', 1.0)
('141790779_썬포토정품GOPOLESNAPPOD스냅포드블루투스셀카봉미니삼각대(리모컨포함)', 1.0)
(

---

## # 이외의 sampling 에서의 결과

In [22]:
prod_id_list.head()

,id,pid
0,"602610157_쿠폰추가★11개구매시쿠폰가8,170원!한끼대용간편선식국내산100%...",602610157
1,600515938_[실라리안]국내산돼지고기로만든가성비갑만두모음!,600515938
2,600185944_[실라리안]100%우리콩안동낫또골라담기,600185944
3,"601529111_닥터딥민감성피부관리여드름성피부,아기피부,수부지피부",601529111
4,600016493_[쇼핑의모든것]대천갓바위김도시락김120봉가격특가,600016493


In [26]:
sampled_prod = prod_id_list.id.sample( 10 ).values

In [28]:
for prod in sampled_prod:
    try:
        #print(prod)
        print('***  id : {}'.format( prod ))
        for similar in model.wv.most_similar( prod ):
            print(similar)
        print('\n\n')
    except:
        pass

***  id : 152372450_[무료배송]모듬튀김10종(600g+600g)1세트
('231977536_해마로해쉬브라운1.2kgx2개/감자튀김/감자패티/간식', 1.0000001192092896)
('330591024_냉동감자튀김,냉동감자,슬라이스감자,얇은감자튀김,감자튀김,맛있는감자튀김,바삭한감자튀김,대용량감자튀김,업소용감자튀김,일자감자튀김', 1.0000001192092896)
('101158444_사옹원우유튀김1kg(드라이아이스)', 1.0000001192092896)
('334818099_오양7299엄선된원료로만든반찬동그랑땡1kg', 1.0000001192092896)
('231945923_해마로슈스트링2kg/막대감자/감자튀김', 1.0000001192092896)
('100094176_스마일감자1.8kg', 1.0000001192092896)
('320900642_오뚜기김말이1kg(아이스박스포장)', 1.0000001192092896)
('156584343_마니커F&G공식몰도톰한입산적구이1kg', 1.0000001192092896)
('271467036_(냉동)랜시생선까스3kg돈까스생선가스업소용돈까스대용량생선가스생선까스', 1.0000001192092896)
('150724897_[Mccain]맛감자2kg_1팩', 1.0000001192092896)



***  id : 101114020_애견드라이기가정용/업소용에어탱크구매대행


/home/wmind/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


('275510120_아테로케라틴바이탈컨디셔너100ml말티즈시츄', 1.0)
('248846352_도그마네킹(견체)+도그위그회색+부분위그/독위그/애견미용연습용위그', 1.0)
('258491747_프라임독실키앤마일드중장모견맞춤애견샴푸300ml', 1.0)
('347211968_벨버드샴푸린스4L(백모용)대용량샴푸애견샴푸펫', 1.0)
('117316351_벨버드백모용샴푸&린스400ml', 1.0)
('131668566_스파샴푸화이트코트백모용강아지샴푸16oz/473ml', 1.0)
('269570887_블링켄(BK-309)부분이발기칼날ㅣ애견이발기칼날ㅣ애완이발기칼날ㅣ부분이발기칼날ㅣ강아지이발기ㅣ고양이이발기', 1.0)
('258491075_프라임독스페셜케어단모견전용애견샴푸500ml', 1.0)
('130523726_램버트케이2in1오트밀베이킹소다컨디셔닝샴푸533ml', 1.0)
('183148133_(현대Hmall)도기맨발톱깍기(프로)HS64애견이발기', 1.0)



***  id : 140845073_[AK몰][훠링]닥터에어컨히터강력탈취제NoLPG
('120754665_그레이드크리스탈로맨스리필(쟈스민가든/한국존슨)', 0.9606130123138428)
('601151929_강아지제습인형/탈취인형모음', 0.9593223333358765)
('341349849_GnJ리얼특별한고양이활성탄미니캣제습인형제습제제습기방향제탈취인형공기정화인형', 0.959276556968689)
('217179273_990)차량용휴지통/차량용품햇빛가리개세차용품차량목쿠션차량팔받침쿠션차량틈새쿠션차량수납함트렁크정리함타이어청소', 0.9588727355003357)
('170547582_냄새먹는강아지고양이차량용탈취제활성탄탈취제공기정화차량냄새제거구매대행', 0.9586650133132935)
('346462004_자동차차량욕실화장실유리거울김서림방지제코팅성에제거습기제거제방지', 0.9585803747177124)
('258461864_피센휴대용공기청정기-차량용개인용공기청정기인테리어공기청정기자동차공기청

---